In [119]:
import requests
import json
import os
import pandas as pd
import numpy as np
import asyncio

api = r"6975761b230fc0d23e1aad6e39b4ef3555cb541f"

In [18]:
census_data_discovery = "https://api.census.gov/data.json"
r = requests.get(census_data_discovery, verify=False)
if r.status_code == 200:
    cdd = r.json()

print(cdd)

/home/aweaver/work/predictables/.env/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.census.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'@context': 'https://project-open-data.cio.gov/v1.1/schema/catalog.jsonld', '@id': 'http://api.census.gov/data/1999/nonemp.json', '@type': 'dcat:Catalog', 'conformsTo': 'https://project-open-data.cio.gov/v1.1/schema', 'describedBy': 'https://project-open-data.cio.gov/v1.1/schema/catalog.json', 'dataset': [{'c_vintage': 1999, 'c_dataset': ['nonemp'], 'c_geographyLink': 'http://api.census.gov/data/1999/nonemp/geography.json', 'c_variablesLink': 'http://api.census.gov/data/1999/nonemp/variables.json', 'c_tagsLink': 'http://api.census.gov/data/1999/nonemp/tags.json', 'c_examplesLink': 'http://api.census.gov/data/1999/nonemp/examples.json', 'c_groupsLink': 'http://api.census.gov/data/1999/nonemp/groups.json', 'c_sorts_url': 'http://api.census.gov/data/1999/nonemp/sorts.json', 'c_documentationLink': 'http://www.census.gov/developer/', 'c_isAggregate': True, 'c_isCube': True, 'c_isAvailable': True, '@type': 'dcat:Dataset', 'title': '1999 Nonemployer Statistics: Non Employer Statistics', 'acc

In [120]:
census_data_sources = pd.concat(
    [pd.Series(cdd["dataset"][i]) for i in range(len(cdd["dataset"]))], axis=1
).T
census_data_sources.columns = [
    col.replace("c_", "").replace("Link", "_link").replace("Code", "_code")
    for col in census_data_sources.columns
]

census_data_sources = census_data_sources.loc[
    census_data_sources.accessLevel.eq("public")
]
census_data_sources["is_aggregate"] = census_data_sources["isAggregate"].fillna(False)
census_data_sources["link_stem"] = (
    census_data_sources["variables_link"]
    .str.split("/")
    .str[:-1]
    .apply(lambda x: "/".join(x))
)

census_data_sources["geography_link"] = (
    census_data_sources["geography_link"].str.split("/").str[-1]
)
census_data_sources["variables_link"] = (
    census_data_sources["variables_link"].str.split("/").str[-1]
)
census_data_sources["tags_link"] = (
    census_data_sources["tags_link"].str.split("/").str[-1]
)
census_data_sources["examples_link"] = (
    census_data_sources["examples_link"].str.split("/").str[-1]
)
census_data_sources["groups_link"] = (
    census_data_sources["groups_link"].str.split("/").str[-1]
)
census_data_sources["sorts_link"] = (
    census_data_sources["sorts_url"].str.split("/").str[-1]
)

census_data_sources["spatial"] = (
    census_data_sources["spatial"]
    .str.replace("US", "United States")
    .str.replace("United Sates", "United States")
)

census_data_sources.drop(
    columns=[
        "license",
        "references",
        "isMicrodata",
        "isTimeseries",
        "contactPoint",
        "distribution",
        "accessLevel",
        "isAggregate",
        "isCube",
        "isAvailable",
        "documentation_link",
        "@type",
        "sorts_url",
    ],
    inplace=True,
)

census_data_sources = census_data_sources.loc[
    ~census_data_sources.spatial.isin(
        [
            "Puerto Rico",
            "American Samoa",
            "Guam",
            "Commonwealth of the Northern Mariana Islands",
            "U.S. Virgin Islands",
        ]
    )
]


# census_data_sources['variables'] = census_data_sources['link_stem variables_link'.split()].apply(lambda x: requests.get(f"https://api.census.gov/data/{x[0]}/{x[1]}.json", verify=False).content, axis=1)


census_data_sources.reset_index(drop=True, inplace=True)
census_data_sources.head()

vintage   dataset  geography_link  variables_link  tags_link  examples_link  \
0    1999  [nonemp]  geography.json  variables.json  tags.json  examples.json   
1    1997  [nonemp]  geography.json  variables.json  tags.json  examples.json   
2    1986     [cbp]  geography.json  variables.json  tags.json  examples.json   
3    1997     [zbp]  geography.json  variables.json  tags.json  examples.json   
4    1987     [cbp]  geography.json  variables.json  tags.json  examples.json   

   groups_link                                              title bureau_code  \
0  groups.json  1999 Nonemployer Statistics: Non Employer Stat...    [006:07]   
1  groups.json  1997 Nonemployer Statistics: Non Employer Stat...    [006:07]   
2  groups.json   1986 County Business Patterns: Business Patterns    [006:07]   
3  groups.json  1997 County Business Patterns - Zip Code Busin...    [006:07]   
4  groups.json   1987 County Business Patterns: Business Patterns    [006:07]   

                                         description  \
0  Nonemployer Statistics is an annual series tha...   
1  Nonemployer Statistics is an annual series tha...   
2  County Business Patterns (CBP) is an annual se...   
3  ZIP Code Business Patterns (ZBP) is an annual ...   
4  County Business Patterns (CBP) is an annual se...   

                                   identifier   keyword    modified  \
0    http://api.census.gov/data/id/NONEMP1999  [census]  2017-02-09   
1    http://api.census.gov/data/id/NONEMP1997  [census]  2017-02-09   
2       http://api.census.gov/data/id/CBP1986  [census]  2019-02-13   
3  http://api.census.gov/data/id/ZBPTotal1997  [census]  2018-01-25   
4       http://api.census.gov/data/id/CBP1987  [census]  2019-02-13   

  program_code        spatial   temporal  \
0    [006:007]  United States  1999/1999   
1    [006:007]  United States  1997/1997   
2    [006:007]  United States  1986/1986   
3    [006:007]  United States  1997/1997   
4    [006:007]  United States  1987/1987   

                                           publisher  is_aggregate  \
0  {'@type': 'org:Organization', 'name': 'U.S. Ce...          True   
1  {'@type': 'org:Organization', 'name': 'U.S. Ce...          True   
2  {'@type': 'org:Organization', 'name': 'U.S. Ce...          True   
3  {'@type': 'org:Organization', 'name': 'U.S. Ce...          True   
4  {'@type': 'org:Organization', 'name': 'U.S. Ce...          True   

                                link_stem  sorts_link  
0  http://api.census.gov/data/1999/nonemp  sorts.json  
1  http://api.census.gov/data/1997/nonemp  sorts.json  
2     http://api.census.gov/data/1986/cbp  sorts.json  
3     http://api.census.gov/data/1997/zbp  sorts.json  
4     http://api.census.gov/data/1987/cbp  sorts.json

In [125]:
async def get_variables_async(link_stem_list_of_dicts):
    """Takes a list of dicts with name and link_stem keys and returns a list of dicts with name, link_stem, variables, geography, tags, examples, and groups keys. Uses asyncio to make requests asynchronously."""

    async def get_variable(link_stem):
        return requests.get(f"{link_stem}/variables.json", verify=False).content

    async def get_geography(link_stem):
        return requests.get(f"{link_stem}/geography.json", verify=False).content

    async def get_tags(link_stem):
        return requests.get(f"{link_stem}/tags.json", verify=False).content

    async def get_examples(link_stem):
        return requests.get(f"{link_stem}/examples.json", verify=False).content

    async def get_groups(link_stem):
        return requests.get(f"{link_stem}/groups.json", verify=False).content

    async def get_sorts(link_stem):
        return requests.get(f"{link_stem}/sorts.json", verify=False).content

    async def get_all(link_stem):
        return {
            "link_stem": link_stem,
            "variables": await get_variable(link_stem),
            "geography": await get_geography(link_stem),
            "tags": await get_tags(link_stem),
            "examples": await get_examples(link_stem),
            "groups": await get_groups(link_stem),
            "sorts": await get_sorts(link_stem),
        }

    async def get_all_wrapper(link_stem):
        return await get_all(link_stem)

    async def get_all_wrapper_wrapper(link_stem_list_of_dicts):
        return await asyncio.gather(
            *[get_all_wrapper(link_stem) for link_stem in link_stem_list_of_dicts]
        )

    return await get_all_wrapper_wrapper(link_stem_list_of_dicts)

In [124]:
list_of_dicts = (
    census_data_sources["link_stem"].apply(lambda x: {"link_stem": x}).tolist()
)
list_of_dicts

[{'link_stem': 'http://api.census.gov/data/1999/nonemp'},
 {'link_stem': 'http://api.census.gov/data/1997/nonemp'},
 {'link_stem': 'http://api.census.gov/data/1986/cbp'},
 {'link_stem': 'http://api.census.gov/data/1997/zbp'},
 {'link_stem': 'http://api.census.gov/data/1987/cbp'},
 {'link_stem': 'http://api.census.gov/data/1999/zbp'},
 {'link_stem': 'http://api.census.gov/data/1988/cbp'},
 {'link_stem': 'http://api.census.gov/data/2000/cbp'},
 {'link_stem': 'http://api.census.gov/data/1989/cbp'},
 {'link_stem': 'http://api.census.gov/data/1990/cbp'},
 {'link_stem': 'http://api.census.gov/data/2000/nonemp'},
 {'link_stem': 'http://api.census.gov/data/1995/cbp'},
 {'link_stem': 'http://api.census.gov/data/1990/pep/int_charagegroups'},
 {'link_stem': 'http://api.census.gov/data/1990/pep/int_natcivpop'},
 {'link_stem': 'http://api.census.gov/data/1996/cbp'},
 {'link_stem': 'http://api.census.gov/data/1990/pep/int_natresafo'},
 {'link_stem': 'http://api.census.gov/data/1995/zbp'},
 {'link_st

In [128]:
# vars = get_variables_async(list_of_dicts)
# vars

vars = census_data_sources["link_stem"].apply(
    lambda x: requests.get(f"{x}/variables.json", verify=False).content
)
vars

/home/aweaver/work/predictables/.env/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.census.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/aweaver/work/predictables/.env/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.census.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/aweaver/work/predictables/.env/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.census.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warning

KeyboardInterrupt: 

In [106]:
url = (
    census_data_sources.loc[census_data_sources.spatial.eq("School Districts")]
    .apply(lambda x: x["link_stem"] + x["variables_link"], axis=1)
    .iloc[0]
)

r = requests.get(url, verify=False)
if r.status_code == 200:
    census_vars = r.json()

census_vars

/home/aweaver/work/predictables/.env/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.census.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'variables': {'for': {'label': "Census API FIPS 'for' clause",
   'concept': 'Census API Geography Specification',
   'predicateType': 'fips-for',
   'group': 'N/A',
   'limit': 0,
   'predicateOnly': True},
  'in': {'label': "Census API FIPS 'in' clause",
   'concept': 'Census API Geography Specification',
   'predicateType': 'fips-in',
   'group': 'N/A',
   'limit': 0,
   'predicateOnly': True},
  'ucgid': {'label': 'Uniform Census Geography Identifier clause',
   'concept': 'Census API Geography Specification',
   'predicateType': 'ucgid',
   'group': 'N/A',
   'limit': 0,
   'predicateOnly': True,
   'hasGeoCollectionSupport': True},
  'time': {'label': 'ISO-8601 Date/Time value',
   'concept': 'Census API Date/Time Specification',
   'required': 'true',
   'predicateType': 'datetime',
   'group': 'N/A',
   'limit': 0,
   'predicateOnly': True,
   'datetime': {'year': True}},
  'SAEPOVRAT5_17RV_PT': {'label': 'Relevant Age 5 to 17 Poverty Ratio Estimate',
   'predicateType': 'floa

In [111]:
census_vars["variables"]

{'for': {'label': "Census API FIPS 'for' clause",
  'concept': 'Census API Geography Specification',
  'predicateType': 'fips-for',
  'group': 'N/A',
  'limit': 0,
  'predicateOnly': True},
 'in': {'label': "Census API FIPS 'in' clause",
  'concept': 'Census API Geography Specification',
  'predicateType': 'fips-in',
  'group': 'N/A',
  'limit': 0,
  'predicateOnly': True},
 'ucgid': {'label': 'Uniform Census Geography Identifier clause',
  'concept': 'Census API Geography Specification',
  'predicateType': 'ucgid',
  'group': 'N/A',
  'limit': 0,
  'predicateOnly': True,
  'hasGeoCollectionSupport': True},
 'time': {'label': 'ISO-8601 Date/Time value',
  'concept': 'Census API Date/Time Specification',
  'required': 'true',
  'predicateType': 'datetime',
  'group': 'N/A',
  'limit': 0,
  'predicateOnly': True,
  'datetime': {'year': True}},
 'SAEPOVRAT5_17RV_PT': {'label': 'Relevant Age 5 to 17 Poverty Ratio Estimate',
  'predicateType': 'float',
  'group': 'N/A',
  'limit': 0},
 'YEA

In [58]:
census_data_sources.columns.tolist()

['vintage',
 'dataset',
 'geography_link',
 'variables_link',
 'tags_link',
 'examples_link',
 'groups_link',
 'sorts_url',
 'title',
 'bureau_code',
 'description',
 'identifier',
 'keyword',
 'modified',
 'program_code',
 'spatial',
 'temporal',
 'publisher',
 'is_aggregate']

In [74]:
census_data_sources.loc[
    census_data_sources.vintage.ge(2015)
    & census_data_sources.title.str.contains("Basic Monthly"),
    census_data_sources.columns.to_series().str.contains("_link"),
]

geography_link  \
635   http://api.census.gov/data/2015/cps/basic/apr/...   
636   http://api.census.gov/data/2015/cps/basic/aug/...   
637   http://api.census.gov/data/2015/cps/basic/dec/...   
638   http://api.census.gov/data/2015/cps/basic/feb/...   
639   http://api.census.gov/data/2015/cps/basic/jan/...   
...                                                 ...   
1356  http://api.census.gov/data/2023/cps/basic/aug/...   
1371  http://api.census.gov/data/2023/cps/basic/sep/...   
1383  http://api.census.gov/data/2023/cps/basic/oct/...   
1388  http://api.census.gov/data/2023/cps/basic/nov/...   
1390  http://api.census.gov/data/2023/cps/basic/dec/...   

                                         variables_link  \
635   http://api.census.gov/data/2015/cps/basic/apr/...   
636   http://api.census.gov/data/2015/cps/basic/aug/...   
637   http://api.census.gov/data/2015/cps/basic/dec/...   
638   http://api.census.gov/data/2015/cps/basic/feb/...   
639   http://api.census.gov/data/2015/cps/basic/jan/...   
...                                                 ...   
1356  http://api.census.gov/data/2023/cps/basic/aug/...   
1371  http://api.census.gov/data/2023/cps/basic/sep/...   
1383  http://api.census.gov/data/2023/cps/basic/oct/...   
1388  http://api.census.gov/data/2023/cps/basic/nov/...   
1390  http://api.census.gov/data/2023/cps/basic/dec/...   

                                              tags_link  \
635   http://api.census.gov/data/2015/cps/basic/apr/...   
636   http://api.census.gov/data/2015/cps/basic/aug/...   
637   http://api.census.gov/data/2015/cps/basic/dec/...   
638   http://api.census.gov/data/2015/cps/basic/feb/...   
639   http://api.census.gov/data/2015/cps/basic/jan/...   
...                                                 ...   
1356  http://api.census.gov/data/2023/cps/basic/aug/...   
1371  http://api.census.gov/data/2023/cps/basic/sep/...   
1383  http://api.census.gov/data/2023/cps/basic/oct/...   
1388  http://api.census.gov/data/2023/cps/basic/nov/...   
1390  http://api.census.gov/data/2023/cps/basic/dec/...   

                                          examples_link  \
635   http://api.census.gov/data/2015/cps/basic/apr/...   
636   http://api.census.gov/data/2015/cps/basic/aug/...   
637   http://api.census.gov/data/2015/cps/basic/dec/...   
638   http://api.census.gov/data/2015/cps/basic/feb/...   
639   http://api.census.gov/data/2015/cps/basic/jan/...   
...                                                 ...   
1356  http://api.census.gov/data/2023/cps/basic/aug/...   
1371  http://api.census.gov/data/2023/cps/basic/sep/...   
1383  http://api.census.gov/data/2023/cps/basic/oct/...   
1388  http://api.census.gov/data/2023/cps/basic/nov/...   
1390  http://api.census.gov/data/2023/cps/basic/dec/...   

                                            groups_link  
635   http://api.census.gov/data/2015/cps/basic/apr/...  
636   http://api.census.gov/data/2015/cps/basic/aug/...  
637   http://api.census.gov/data/2015/cps/basic/dec/...  
638   http://api.census.gov/data/2015/cps/basic/feb/...  
639   http://api.census.gov/data/2015/cps/basic/jan/...  
...                                                 ...  
1356  http://api.census.gov/data/2023/cps/basic/aug/...  
1371  http://api.census.gov/data/2023/cps/basic/sep/...  
1383  http://api.census.gov/data/2023/cps/basic/oct/...  
1388  http://api.census.gov/data/2023/cps/basic/nov/...  
1390  http://api.census.gov/data/2023/cps/basic/dec/...  

[108 rows x 5 columns]

In [68]:
req = requests.get("https://api.census.gov/data/id/CPSBASIC201504")
req.content

b'<!doctype html><html lang="en"><head><title>HTTP Status 404 ? Not Found</title><style type="text/css">body {font-family:Tahoma,Arial,sans-serif;} h1, h2, h3, b {color:white;background-color:#525D76;} h1 {font-size:22px;} h2 {font-size:16px;} h3 {font-size:14px;} p {font-size:12px;} a {color:black;} .line {height:1px;background-color:#525D76;border:none;}</style></head><body><h1>HTTP Status 404 ? Not Found</h1></body></html>'

In [10]:
cdd["dataset"][0]["c_geographyLink"]

'http://api.census.gov/data/1999/nonemp/geography.json'